## Active fire detection using Sentinel-3 SLSTR data - descending acquisitions

In [ ]:
import os
import sys
import html
sys.path.append(os.getcwd())
from active_fire_helpers import *
import datetime
import shutil
from py_snap_helpers import snap_graph, get_operator_default_parameters
os.environ['_JAVA_OPTIONS'] = '-Xms1g -Xmx1g'

In [ ]:
%load_ext autoreload
%autoreload 2

### Input product

Set the input identifier and the path of the unzipped product:

In [ ]:
input_identifier = 'S3A_SL_1_RBT____20170618T104548_20170618T104848_20181004T040944_0179_019_051______LR1_R_NT_003'

data_path = '/workspace/data'

# For visual check:
local_path = os.path.join(data_path, input_identifier + '.SEN3', 'xfdumanifest.xml')
print(local_path)

The following checks whether the manifest file exists in the configured location (if not, adjust the values in the previous cell):

In [ ]:
if not os.path.exists(local_path): raise BaseException('Check the README to get the EO data')

### Parameters

Set the parameters for the algorithm. Adjust, if necessary, the coordinates:

In [ ]:
aoi_wkt = 'POLYGON((-8.72 39.422,-8.72 40.191,-7.162 40.191,-7.162 39.422,-8.72 39.422))'

f1_bt_day_threshold = 325

bt_diff_day_threshold = 18

f1_bt_night_threshold = 315

bt_diff_night_threshold = 15

### Sentinel-3 pre-processing - day acquisition

Update SNAP Graph parameters:

In [ ]:
read = dict()
read['file'] =  local_path
read['formatName'] = 'Sen3_SLSTRL1B_500m'

rad2refl = dict()
rad2refl['sensor'] = 'SLSTR_500m'
rad2refl['copyTiePointGrids'] = 'true'
rad2refl['copyFlagBandsAndMasks'] = 'true'
rad2refl['copyNonSpectralBands'] = 'true'

resample = dict()
resample['referenceBandName'] = 'F1_BT_in'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

subset = dict()
subset['geoRegion'] = aoi_wkt
subset['fullSwath'] = 'false'
subset['copyMetadata'] = 'true'

write = dict()
write['file'] = 'temp'

Run the pre-processing and wait until it completed (this may take a short while):

In [ ]:
pre_processing(orbit_direction='DESCENDING',
               Read=read, 
               Rad2Refl=rad2refl,
               Resample=resample,
               Reproject=reproject,
               Subset=subset,
               Write=write)

### Active fire

Band arithmetic expressions:

In [ ]:
cloud_mask_expression = html.escape('(S2_reflectance_an + S3_reflectance_an) > 0.9 or S9_BT_in < 265 or ((S2_reflectance_an + S3_reflectance_an) > 0.7 and S9_BT_in < 285)')

active_fire_expression = html.escape('F1_BT_in > {0} and (F1_BT_in - F2_BT_in) > {1}').format(f1_bt_day_threshold, bt_diff_day_threshold)

### Active fire detection

In [ ]:
output_name = 'active_fire_{0}.tif'.format(input_identifier)

In [ ]:
read = dict()

read['file'] = 'temp.dim'

bands = '''<targetBands>
        <targetBand>
          <name>active_fire_detected</name>
          <type>float32</type>
          <expression>{0} ? 0 : {1} ? 1 : 0</expression>
          <description/>
          <unit/>
          <noDataValue>NaN</noDataValue>
        </targetBand>
        <targetBand>
          <name>cloud</name>
          <type>float32</type>
          <expression>{0}</expression>
          <description/>
          <unit/>
          <noDataValue>NaN</noDataValue>
        </targetBand>
         <targetBand>
          <name>reflectance_mask</name>
          <type>float32</type>
          <expression>S3_reflectance_an == -10000 || S2_reflectance_an == -10000 || S1_reflectance_an == -10000 ? 0 : 1</expression>
          <description/>
          <unit/>
          <noDataValue>NaN</noDataValue>
        </targetBand>
        <targetBand>
          <name>S3_reflectance_an</name>
          <type>float32</type>
          <expression>S3_reflectance_an</expression>
          <description/>
          <unit/>
          <noDataValue>NaN</noDataValue>
        </targetBand>
          <targetBand>
          <name>S2_reflectance_an</name>
          <type>float32</type>
          <expression>S2_reflectance_an</expression>
          <description/>
          <unit/>
          <noDataValue>NaN</noDataValue>
        </targetBand>
          <targetBand>
          <name>S1_reflectance_an</name>
          <type>float32</type>
          <expression>S1_reflectance_an</expression>
          <description/>
          <unit/>
          <noDataValue>NaN</noDataValue>
        </targetBand>
        </targetBands>'''.format(cloud_mask_expression, 
                                 active_fire_expression)

band_maths = dict()
band_maths['targetBandDescriptors'] = bands 

write = dict()
write['file'] = output_name
write['formatName'] = 'GeoTIFF-BigTiff'

Run the active fire processing and wait until it completed (this may take a short while):

In [ ]:
active_fire(Read=read,
            BandMaths=band_maths,
            Write=write)

Cleanup:

In [ ]:
os.remove('temp.dim')

shutil.rmtree('temp.data')

**END**